In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/one_dataset_v2.csv')

In [ ]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'title', 'authors',
       'n_citation', 'doi', 'url', 'pdf', 'conf', 'fos', 'year', 'pool',
       'topic', 'labels', 'bib name', 'dIDs', 'ddIDs'],
      dtype='object')

## direction base to der to derofder
## So row is src and col is dest I think

In [ ]:
def flatten(l):
  return [x for y in l for x in y]

def str2list(s,pattern=' #;# '):
  s = s.split(pattern)
  s = [x.strip() for x in s]

  s = [x.strip("'") for x in s]
  s = [x.strip('"') for x in s]
  return(s)


def make_adjmat(row):
  src = [row['Unnamed: 0']]
  ds = str2list(row['dIDs'])
  dds = str2list(row['ddIDs'])
  assert len(ds)==len(dds)
  
  dds = [str2list(x,' #$# ') for x in dds]

  all_papers = set(src + ds+ flatten(dds))
  ind = dict()

  i = 0
  for pap in all_papers:
    ind[pap] = i
    i+=1
  
  sparse = []
  sparse_ind = []

  for d in ds:
    sparse.append((src[0],d))
    sparse_ind.append((ind[src[0]],ind[d]))
  
  for i in range(len(ds)):
    d = ds[i]
    dd = dds[i]
    for derofder in dd:
      sparse.append((d,derofder))
      sparse_ind.append((ind[d],ind[derofder]))

  mat = np.zeros((len(all_papers),len(all_papers)),dtype=np.int32)
  mat = mat.tolist()
  for tup in sparse_ind:
    mat[tup[0]][tup[1]] = 1
  # print(mat)

  df = pd.DataFrame(mat, index=all_papers, columns=all_papers)

  return df

In [ ]:
dict_init_heats = {'bIDs':[],'heat':[]}
for index,row in df.iterrows():
  if (isinstance(row['dIDs'],float)):
    continue
  else:
    try:
      a = make_adjmat(row)
      a.to_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/Influence_score/adj_mat/'+str(row['Unnamed: 0'])+'.csv')

      dict_init_heats['bIDs'].append(row['Unnamed: 0'])
      dict_init_heats['heat'].append(len(str2list(row['dIDs'])))
    except:
      pass


In [ ]:
len(dict_init_heats['bIDs'])

3697

In [ ]:
df = pd.DataFrame(dict_init_heats)
df.to_csv('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/Influence_score/initial_heats.csv')

In [ ]:
!mv '/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/adj_mat' '/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/Influence_score/adj_mat'

In [ ]:
import os
indices_w_graph = os.listdir('/content/drive/MyDrive/SAHA PROJECT - connectedpapers zip files/Influence_score/adj_mat')
indices_w_graph = [int(x.split('.')[0]) for x in indices_w_graph]
len(indices_w_graph)

3697